In [1]:
import os
import json
import cv2
from PIL import Image
from tqdm import tqdm
from glob import glob

import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset
from torchvision import transforms

'''
{'filename': '5f656a0f627a3ef96dec882437e3e7ada1c7a877201cf54dcd7a2c4508588ff3_여_30_기쁨_공공시설&종교&의료시설_20201204105732-001-007.jpg',
 'gender': '여',
 'age': 30,
 'isProf': '전문인',
 'faceExp_uploader': '기쁨',
 'bg_uploader': '공공시설/종교/의료시설',
 'annot_A': {'boxes': {'maxX': 1912.2253,
   'maxY': 1581.6027,
   'minX': 1187.4949,
   'minY': 579.22235},
  'faceExp': '기쁨',
  'bg': '공공시설/종교/의료'},
 'annot_B': {'boxes': {'maxX': 1912.348108621648,
   'maxY': 1572.1522585800617,
   'minX': 1206.363701502596,
   'minY': 579.1777983055337},
  'faceExp': '기쁨',
  'bg': '공공시설/종교/의료'},
 'annot_C': {'boxes': {'maxX': 1890.909447114109,
   'maxY': 1567.448627450284,
   'minX': 1183.8414475546967,
   'minY': 596.9434661684523},
  'faceExp': '기쁨',
  'bg': '공공시설/종교/의료'}}
'''

In [ ]:
class BaegDataset(Dataset):
    def __init__(self ,mode = 'train', transform = None):
        self.mode = mode
        # image dataset
        # image dataset 병합
        upset_list = glob("/data/Emotion_data/Training/upset/*")
        pleasure_list = glob("/data/Emotion_data/Training/pleasure/*")
        hurt_list = glob("/data/Emotion_data/Training/hurt/*")
        anger_list = glob("/data/Emotion_data/Training/anger/*")
        unrest_list = glob("/data/Emotion_data/Training/unrest/*")
        sad_list = glob("/data/Emotion_data/Training/sad/*")
        neutrality_list = glob("/data/Emotion_data/Training/neutrality/*")
        self.data_list = upset_list + pleasure_list + anger_list + unrest_list + sad_list + neutrality_list
        
        # json dataset
        self.label_list = glob("/data/Emotion_data/Training/label/*")
    
    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        # load images and mask
        img_path = self.data_list[idx]
        img = Image.open(img_path).convert("RGB")
        img = np.array(img, np.float32)
        
        label = 
        if self.transform  is not None:
            img = self.transform(img)
            
        # 1. filename만 따로 빼서 for문 돌려서 json_list에 있는 것과 비교
        img_name = img_path.split('/')
        for json_list in label_list:
            with open(json_list, 'r') as f:
                json_data = json.load(f)
                for i in range(0, len(json_data)):
                    filename = json_data[i]['filename']
                    if filename == img_name:
                        mask = json_data[i]
                        
        
        # area : box의 면적으로써 나중에 IOU구하려고 만든거.
        x_min = mask['annot_A']['boxes']['minX']
        x_max = mask['annot_A']['boxes']['maxX']
        y_min = mask['annot_A']['boxes']['minY']
        y_max = mask['annot_A']['boxes']['maxY']
        boxes = [x_min, y_min, x_max, y_max]
        boxes = torch.as_tensor(boxes, dtype = torch.float32)
        
        area = (boxes[3] - boxes[1]) * (boxes[2] - boxes[0])